In [3]:

import anndata as ad
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd

from typing import List, Optional, Callable

In [5]:
# load the fuckin data

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

train_latent_path = "/dtu/blackhole/06/213542/adata_preprocessed_subset.h5ad"
flow_model_save_path = "/dtu/blackhole/06/213542/flow_model.pt"

Using device: cpu


In [ ]:
# ---- Load AnnData with latent space ----
adata = ad.read_h5ad(train_latent_path)

assert "X_latent" in adata.obsm, "Latent representation X_latent not found in .obsm"
z1 = adata.obsm["X_latent"]              # shape [N, 50]

# cell type labels
assert "cell_type" in adata.obs.columns, "cell_type column not found in adata.obs"
celltypes = pd.Categorical(adata.obs["cell_type"])
y_idx = torch.tensor(celltypes.codes, dtype=torch.long)   # integer labels
num_celltypes = len(celltypes.categories)

# library size (n_counts)
assert "n_counts" in adata.obs.columns, "n_counts column not found in adata.obs"
libsize = adata.obs["n_counts"].values.astype(np.float32)
log_l = np.log(libsize + 1.0)[:, None]    # shape [N, 1]

# ---- Convert to torch tensors ----
z1 = torch.tensor(z1, dtype=torch.float32)
log_l = torch.tensor(log_l, dtype=torch.float32)

# one-hot for cell types
y_onehot = F.one_hot(y_idx, num_classes=num_celltypes).float()

print("z1 shape:", z1.shape)
print("y_onehot shape:", y_onehot.shape)
print("log_l shape:", log_l.shape)
print("Num cell types:", num_celltypes)

In [ ]:
class LatentFlowDataset(Dataset):
    def __init__(self, z, y_onehot, log_l):
        assert len(z) == len(y_onehot) == len(log_l)
        self.z = z
        self.y = y_onehot
        self.log_l = log_l

    def __len__(self):
        return self.z.shape[0]

    def __getitem__(self, idx):
        return self.z[idx], self.y[idx], self.log_l[idx]

batch_size = 512

dataset = LatentFlowDataset(z1, y_onehot, log_l)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

len(dataset), len(dataloader)

In [ ]:
class LatentFlowDataset(Dataset):
    def __init__(self, z, y_onehot, log_l):
        assert len(z) == len(y_onehot) == len(log_l)
        self.z = z
        self.y = y_onehot
        self.log_l = log_l

    def __len__(self):
        return self.z.shape[0]

    def __getitem__(self, idx):
        return self.z[idx], self.y[idx], self.log_l[idx]

batch_size = 512

dataset = LatentFlowDataset(z1, y_onehot, log_l)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

len(dataset), len(dataloader)

In [ ]:
# -------------------------------
# MLP (copied from autoencoder)
# -------------------------------
class MLP(nn.Module):
    def __init__(self, 
                 dims: List[int],
                 batch_norm: bool = True, 
                 dropout: bool = True, 
                 dropout_p: float = 0.1, 
                 activation: Optional[Callable] = nn.ELU, 
                 final_activation: Optional[str] = None):
        super().__init__()
        self.dims = dims
        layers = []
        for i in range(len(dims[:-2])):
            block = [nn.Linear(dims[i], dims[i+1])]
            if batch_norm:
                block.append(nn.BatchNorm1d(dims[i+1]))
            block.append(activation())
            if dropout:
                block.append(nn.Dropout(dropout_p))
            layers.append(nn.Sequential(*block))
        layers.append(nn.Linear(dims[-2], dims[-1]))
        self.net = nn.Sequential(*layers)
        if final_activation == "tanh":
            self.final_activation = nn.Tanh()
        elif final_activation == "sigmoid":
            self.final_activation = nn.Sigmoid()
        else:
            self.final_activation = None

    def forward(self, x):
        x = self.net(x)
        return x if self.final_activation is None else self.final_activation(x)

# -------------------------------
# Vector Field Network
# -------------------------------
class VectorField(nn.Module):
    def __init__(self, latent_dim: int, cond_dim: int, hidden_dim: int = 256):
        super().__init__()
        # time embedding
        self.time_mlp = nn.Sequential(
            nn.Linear(1, 64),
            nn.SiLU(),
            nn.Linear(64, 64),
            nn.SiLU(),
        )
        # main network
        self.net = MLP(
            dims=[latent_dim + 64 + cond_dim, hidden_dim, hidden_dim, latent_dim],
            batch_norm=True,
            dropout=True,
            dropout_p=0.1,
            activation=nn.ELU,
        )

    def forward(self, zt, t, cond):
        """
        zt: [B, latent_dim]
        t: [B, 1]
        cond: [B, cond_dim]
        """
        t_emb = self.time_mlp(t)           # [B, 64]
        x = torch.cat([zt, t_emb, cond], dim=1)
        return self.net(x)

NameError: name 'VectorField' is not defined

In [ ]:
latent_dim = z1.shape[1]          # 50
cond_dim = num_celltypes + 1      # cell types + log size factor

vf = VectorField(latent_dim=latent_dim, cond_dim=cond_dim).to(device)
optimizer = torch.optim.AdamW(vf.parameters(), lr=1e-4)

print(vf)

NameError: name 'B' is not defined

In [ ]:
import math

num_epochs = 50             # adjust as you like
drop_prob = 0.15            # probability to drop conditioning (for unconditional training)

vf.train()

for epoch in range(1, num_epochs + 1):
    epoch_loss = 0.0
    n_batches = 0

    for z1_batch, y_batch, logl_batch in dataloader:
        z1_batch = z1_batch.to(device)          # [B, d]
        y_batch = y_batch.to(device)            # [B, num_celltypes]
        logl_batch = logl_batch.to(device)      # [B, 1]

        B, d = z1_batch.shape

        # 1) sample noise z0
        z0 = torch.randn_like(z1_batch)

        # 2) sample time t ~ U(0,1)
        t = torch.rand(B, 1, device=device)

        # 3) straight-line interpolation
        zt = (1.0 - t) * z0 + t * z1_batch

        # 4) target velocity u = z1 - z0 (independent of t)
        u = z1_batch - z0

        # 5) build conditioning vector [onehot, log_l]
        cond_full = torch.cat([y_batch, logl_batch], dim=1)   # [B, num_celltypes+1]

        # 6) classifier-free guidance training: randomly drop conditioning
        mask = (torch.rand(B, 1, device=device) > drop_prob).float()
        cond_train = cond_full * mask    # some rows become all zeros (unconditional)

        # 7) predicted velocity
        pred = vf(zt, t, cond_train)

        loss = ((pred - u) ** 2).mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        n_batches += 1

    avg_loss = epoch_loss / max(1, n_batches)
    print(f"Epoch {epoch}/{num_epochs} - Flow Matching Loss: {avg_loss:.6f}")

# save trained flow model
torch.save(vf.state_dict(), flow_model_save_path)
print(f"Flow model saved to {flow_model_save_path}")